In [ ]:
from enum import unique

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
# import matplotlib as mpl
import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder,StandardScaler
# from  sklearn.feature_extraction.text import CountVectorizer
# from sklearn import metrics
# from xgboost import XGBRegressor
# from datetime import datetime



import warnings

from fontTools.subset import subset

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('Customer_Segmentation.csv')
df.head()


## Finding missing values

In [ ]:
counts_missing = df.isnull().sum()
missing_percentage = (counts_missing / len(df)) * 100

#summary of missing values
missing_summary = pd.DataFrame({
    'Missing Values': counts_missing,
    'Percentage': missing_percentage
}).reset_index().rename(columns={'index': 'Column'})
print(missing_summary)

# Filling missing data
## Income
we have 24 missing values in the Income column, which is significant. We will handle this by using median imputation.

1. Median is robust to outliers, which is important for income data that may have extreme values.
2. It preserves the distribution of the data better than mean imputation.

In [ ]:
#we decided to use median impoutation for the missing values in Income column.This is because of the following reasons

df['Income'].fillna(df['Income'].median(), inplace=True)
print(df["Income"].isnull().sum()) 


## Finding duplicates
No duplicates found

In [ ]:
df.duplicated().sum()

## Date Formatting
Dt_Customer is in the format of 'dd-mm-yyyy', we will convert it to a datetime object for easier analysis.

In [ ]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y', errors='coerce')
print("First 20 enrollment dates:")
print(df['Dt_Customer'].head(20))

print('\nNull vales after formatting: ')
print(df[df['Dt_Customer'].isnull()])

## Checking for constant columns
Constant columns are those that have the same value for all rows. These columns do not provide any useful information for analysis or modeling, so they can be safely removed.

In [ ]:

constant_columns = [col for col in df.columns if df[col].nunique() == 1]
print(constant_columns)


## Dropping constant

### found 2
- AcceptedCmp3
- Z_Revenue

We found 2 constant columns: 'AcceptedCmp3' and 'Z_Revenue'. These columns have the same value for all rows, which means they do not provide any useful information for analysis or modeling. Therefore, we will drop them from the dataset.

In [ ]:

# Remove constant columns
df = df.drop(columns=constant_columns)

print(f"Removed {len(constant_columns)} constant columns: {constant_columns}")

## Finding Outliers
Using the Interquartile Range (IQR) method to identify outliers in numerical columns. The IQR is a measure of statistical dispersion and is used to detect outliers by calculating the range between the first quartile (Q1) and the third quartile (Q3).


In [ ]:
#identifying outliers using IQR method
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Exclude ID and boolean columns
bool_cols = [col for col in df.columns if np.all(np.isin(df[col].unique(), [0, 1]))]
numerical_cols = [col for col in numerical_cols if col not in bool_cols and col != 'ID']
# list of Outlier
def outlier_summary(dataframe):

    outlier_info = []
    total_outliers = 0

    for col in numerical_cols:
        # Calculate IQR
        Q1 = dataframe[col].quantile(0.25)
        Q3 = dataframe[col].quantile(0.75)
        IQR = Q3 - Q1

        # Find outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = dataframe[(dataframe[col] < lower_bound) | (dataframe[col] > upper_bound)]
        outlier_count = len(outliers)

        if outlier_count > 0:
            outlier_percentage = (outlier_count / len(dataframe)) * 100
            outlier_info.append((col, outlier_count, outlier_percentage))
            total_outliers += outlier_count

    print(f"Columns with outliers: {len(outlier_info)}")
    print(f"Total outliers: {total_outliers}")

    if outlier_info:
        print("Columns with outliers:")
        print()
        # Sort by outlier count (descending)
        #outlier_info.sort(key=lambda x: x[1], reverse=True)

        for col, count, percentage in outlier_info:
            print(f"{col}: {count} outliers ({percentage:.1f}%)")
    else:
        print("No outliers detected in any column!")

# Run the outlier summary
outlier_summary(df)


In [ ]:
#visualize the outliers using boxplots
def plot_outliers(dataframe):
    n_cols = 3
    n_plots = len(numerical_cols)
    n_rows = (n_plots + n_cols - 1) // n_cols  # Ceiling division

    plt.figure(figsize=(5 * n_cols, 4 * n_rows))

    for i, col in enumerate(numerical_cols):
        plt.subplot(n_rows, n_cols, i + 1)
        sns.boxplot(x=dataframe[col])
        plt.title(f'Boxplot of {col}')
        plt.xlabel(col)

    plt.tight_layout()
    plt.show()


# Run the outlier visualization
plot_outliers(df)


## handling the outliers
# We will use the capping method to handle outliers. This involves replacing outliers with the nearest non-outlier value, which is determined by the IQR method.


In [ ]:
# Capping method to handle outliers
def cap_outliers(dataframe):

    df_capped = dataframe.copy()

    for col in numerical_cols:
        # Calculate IQR
        Q1 = df_capped[col].quantile(0.25)
        Q3 = df_capped[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Count outliers before capping
        outliers_before = len(df_capped[(df_capped[col] < lower_bound) | (df_capped[col] > upper_bound)])

        # Apply capping
        df_capped[col] = df_capped[col].clip(lower=lower_bound, upper=upper_bound)

        # Count outliers after capping
        outliers_after = len(df_capped[(df_capped[col] < lower_bound) | (df_capped[col] > upper_bound)])

        if outliers_before > 0:
            print(f"{col}: {outliers_before} outliers capped")

    print(f"\nOutlier capping completed!")
    return df_capped #return the capped DataFrame while preserving the original DataFrame

# Apply capping to the dataset
df_capped = cap_outliers(df)





## Visualizatin after capping
# Visualize the capped data using boxplots
We will create boxplots for each numerical column to visualize the effect of capping on outliers. The boxplots will show the distribution of the data and highlight any remaining outliers after capping.

In [ ]:


# Also created a simple boxplot of just the capped data
def plot_capped_data_only(capped_df):

    if 'ID' in numerical_cols:
        numerical_cols.remove('ID')

    n_cols = 3
    n_plots = len(numerical_cols)
    n_rows = (n_plots + n_cols - 1) // n_cols

    plt.figure(figsize=(5 * n_cols, 4 * n_rows))

    for i, col in enumerate(numerical_cols):
        plt.subplot(n_rows, n_cols, i + 1)
        sns.boxplot(x=capped_df[col], color='lightgreen')
        plt.title(f'Capped Data: {col}')
        plt.xlabel(col)

    plt.tight_layout()
    plt.show()

print("Boxplots of capped data only:")
plot_capped_data_only(df_capped)

# Check if the cleaned customer CSV exists
if os.path.exists('cleaned_customer_segmentation.csv'):
    print(f"Existing cleaned_customer_segmentation.csv found, removing it...")
    os.remove('cleaned_customer_segmentation.csv')
    print("File removed successfully.")

# Save the new cleaned dataset
df_capped.to_csv('cleaned_customer_segmentation.csv', index=False)
print(f"New cleaned_customer_segmentation.csv saved successfully ({os.path.getsize('cleaned_customer_segmentation.csv')/1024:.2f} KB)")

# Verify the file was saved correctly
if os.path.exists('cleaned_customer_segmentation.csv'):
    # Load the cleaned dataset to confirm it was saved properly
    df_cleaned = pd.read_csv('cleaned_customer_segmentation.csv')
    print(f"Loaded cleaned dataset with shape: {df_cleaned.shape}")

## Feature Egineering
Feature engineering is the process of using domain knowledge to extract features from raw data that make machine learning algorithms work. In this section, we will create new features based on the existing data in the dataset.

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Load data
df = pd.read_csv('cleaned_customer_segmentation.csv')


## Text encoding
### Education
- we chose ordinal encoding to put into perspective the increasing level of education
1. first we will find the unique eduaction levels

In [ ]:


unique_eduaction_levels= df['Education'].unique()

print(unique_eduaction_levels)

## next steps
2. supply ordinal encoder with oredered categories

`['Basic', '2n Cycle', 'Graduation', 'Master', 'PhD']
`

3. fit the education column

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordered_education_categories = ['Basic', '2n Cycle', 'Graduation', 'Master', 'PhD']
ordinal_encoder = OrdinalEncoder(categories=[ordered_education_categories])

education_encoded = ordinal_encoder.fit_transform(df[['Education']])

df['Education'] = education_encoded

In [ ]:

print('Unique Marital Status:')
print(df['Marital_Status'].unique())
print('\nMarital Status value counts:')
print(df['Marital_Status'].value_counts())

## Marital status evaluation

### Unique Marital Status:
`['Single' 'Together' 'Married' 'Divorced' 'Widow' 'Alone' 'Absurd' 'YOLO']
`
### Marital Status value counts:
`Marital_Status,
Married     864,
Together    580,
Single      480,
Divorced    232,
Widow        77,
Alone         3,
Absurd        2`

## Deduction
### Wrong data
- Yolo alone and absurd are wrong data entries.
- Alone is the same as single and its values can be replaced with single
- Absurd and yolo will be replaced with the mode



In [ ]:
# replace Alone with Single
df['Marital_Status'] = df['Marital_Status'].replace('Alone', 'Single')
print('\nMode: ',df['Marital_Status'].mode()[0])
# Absurd and yolo  replaced with the mode
df['Marital_Status'] = df['Marital_Status'].replace(['YOLO','Absurd'], df['Marital_Status'].mode()[0])

#now check for the current state
print('Unique Marital Status:')
print(df['Marital_Status'].unique())
print('\nMarital Status value counts:')
print(df['Marital_Status'].value_counts())


### marital status encoding
We went with **one hot encoding**,
this is because ~~nominal encoding~~ would create a non existent _hierachy_

In [ ]:
# One-hot encode marital status
marital_dummies = pd.get_dummies(df['Marital_Status'], prefix='Marital')
df = pd.concat([df.drop('Marital_Status', axis=1), marital_dummies], axis=1)

print(df.info())


In [ ]:

# Feature engineering with corrected date parsing
try:

    df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='mixed', dayfirst=True)
except ValueError:

    df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y', errors='coerce')


df['Age'] = 2025- df['Year_Birth']
df['Total_Dependents'] = df['Kidhome'] + df['Teenhome']
df['Is_Parent'] = (df['Total_Dependents'] > 0).astype(int)
df['Total_Spending'] = df[['MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']].sum(axis=1)
df['Total_Purchases'] = df[['NumWebPurchases','NumCatalogPurchases','NumStorePurchases']].sum(axis=1)
df['Average_Spent'] = df['Total_Spending']/df['Total_Purchases'].replace(0,1)
df['Tenure_Days'] = (pd.to_datetime('today') - df['Dt_Customer']).dt.days
df['TotalAcceptedCmp']=df[['AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5']].sum(axis=1)
df['WebPurchaseRatio'] = df['NumWebPurchases'] / df['Total_Purchases'].replace(0, 1)
df['StorePurchaseRatio'] = df['NumStorePurchases'] / df['Total_Purchases'].replace(0, 1)

campaign_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
df['Total_Accepted_Campaigns'] = df[campaign_cols].sum(axis=1)
df['Ever_Accepted_Campaign'] = (df['Total_Accepted_Campaigns'] > 0).astype(int)

# Verify date conversion
print("\nDate conversion samples:")
print(df[['Dt_Customer', 'Tenure_Days']].head())

# Drop unnecessary columns
# df = df.drop(['ID', 'Year_Birth', 'Z_CostContact', 'Z_Revenue','Complain','Response','Kidhome', 'Teenhome','AcceptedCmp1','MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds'], axis=1, errors='ignore')

# Check if the featured customer CSV exists
if os.path.exists('featured_customer_segmentation.csv'):
    print(f"Existing featured_customer_segmentation.csv found, removing it...")
    os.remove('featured_customer_segmentation.csv')
    print("File removed successfully.")

# Save the new featured dataset
df.to_csv('featured_customer_segmentation.csv', index=False)
print(f"New featured_customer_segmentation.csv saved successfully ({os.path.getsize('featured_customer_segmentation.csv')/1024:.2f} KB)")



In [ ]:
df=pd.read_csv('cleaned_customer_segmentation.csv')
print("\nFirst 10 rows with all columns:")
print(df.head(10))

In [ ]:

# Select numerical columns (exclude IDs/constants)
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
cols_to_drop = ['ID','Year_Birth','Dt_Customer','Complain','Response','AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5']
numerical_cols = [col for col in numerical_cols if col not in cols_to_drop]

# Calculate correlations
corr_matrix = df[numerical_cols].corr()

# Set up the plot
plt.figure(figsize=(18, 14))  # Increased size for more columns

# Create heatmap with adjustments
heatmap = sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap='coolwarm',
    center=0,
    linewidths=0.5,
    cbar_kws={'shrink': 0.8},
    annot_kws={'size': 10}  # Smaller annotation font
)

# Rotate and align labels
plt.xticks(
    rotation=45,
    ha='right',
    rotation_mode='anchor'
)
plt.yticks(rotation=0)

# Force tight layout and adjust margins
plt.tight_layout(pad=2.0)  # Extra padding
plt.title('Correlation Heatmap (Adjusted)', fontsize=16, pad=20)
plt.show()

In [ ]:


#this is the heatmap for the featured csv

# Load data
df = pd.read_csv('featured_customer_segmentation.csv')

# Select numerical columns (exclude IDs/constants)
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
cols_to_drop = ['ID','Year_Birth','Dt_Customer','Complain','Response','AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5']
numerical_cols = [col for col in numerical_cols if col not in cols_to_drop]

# Calculate correlations
corr_matrix = df[numerical_cols].corr()

# Set up the plot
plt.figure(figsize=(18, 14))  # Increased size for more columns

# Create heatmap with adjustments
heatmap = sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap='coolwarm',
    center=0,
    linewidths=0.5,
    cbar_kws={'shrink': 0.8},
    annot_kws={'size': 10}  # Smaller annotation font
)

# Rotate and align labels
plt.xticks(
    rotation=45,
    ha='right',
    rotation_mode='anchor'
)
plt.yticks(rotation=0)

# Force tight layout and adjust margins
plt.tight_layout(pad=2.0)  # Extra padding
plt.title('Correlation Heatmap (Adjusted)', fontsize=16, pad=20)
plt.show()

In [ ]:
print(df.columns)

# Clustering based on Spending Patterns
In this section, we will perform clustering on the dataset to identify distinct customer segments based on their spending patterns. We will use KMeans clustering for this purpose.

1. **Feature Selection**: We will select relevant features that represent customer spending patterns.

In [ ]:
spending_features = [
    'MntWines', 'MntFruits', 'MntMeatProducts',
    'MntFishProducts', 'MntSweetProducts', 'MntGoldProds','Total_Spending'
]



2. **Data Preparation**: We will standardize the selected features to ensure that they are on the same scale.

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Load dataset
df = pd.read_csv('featured_customer_segmentation.csv')

# Select relevant spending columns
X = df[spending_features]

# extract ratios for all the spending features
# Create ratio features for spending categories, excluding the Total_Spending itself
spending_categories = [feature for feature in spending_features if feature != 'Total_Spending']
for feature in spending_categories:
    X[f'{feature}_ratio'] = X[feature] / X['Total_Spending'].replace(0, 1)  # Avoid division by zero

df_ratios= X[[f'{feature}_ratio' for feature in spending_categories]]


# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_ratios)

print(df_ratios.head())



3. Optimizing the **number of clusters** using the Elbow Method

In [ ]:
inertia = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

print(inertia)

plt.plot(range(1, 10), inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Spending Pattern Clustering')
plt.grid(True)
plt.show()


4. **Applying KMeans Clustering** with the optimal number of clusters
We will choose 3 clusters based on the elbow method.
this is because the inertia starts to level off after 3 clusters, indicating that adding more clusters does not significantly reduce inertia.

In [ ]:
# Apply KMeans clustering with clusters
kmeans = KMeans(n_clusters=3, random_state=42)
df['Spending_Cluster'] = kmeans.fit_predict(X_scaled)


cluster_summary = df.groupby('Spending_Cluster')[spending_features].mean().round(2)
print(cluster_summary)


5. **Analyzing Cluster Characteristics**

We will analyze the characteristics of each cluster by computing the mean spending for each product category.

We will also visualize the average spending per product category for each cluster using a grouped bar chart.


In [ ]:
# Group by cluster and compute means
cluster_means = df.groupby('Spending_Cluster')[spending_features].mean()

print(cluster_means)

# Transpose for easier plotting (products on x-axis)
cluster_means_T = cluster_means.T

# Plot grouped bar chart
cluster_means_T.plot(kind='bar', figsize=(10, 10))

plt.title('Average Spending per Product Category by Cluster')
plt.xlabel('Product Category')
plt.ylabel('Average Spending')
plt.legend(title='Cluster')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Here is the grouped bar chart showing the average spending per product category for each cluster. Each bar represents the average spending for a specific product category, and different colors represent different clusters. This visualization helps us understand how each cluster allocates its spending across various product categories.

In [ ]:
# Transpose the DataFrame so categories are on the x-axis
cluster_means.T.plot(kind='bar', figsize=(10, 6), colormap='Set1')

plt.title('Average Spending per Product Category by Cluster')
plt.ylabel('Average Amount Spent')
plt.xlabel('Product Category')
plt.xticks(rotation=45)
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()


This is a stacked bar chart showing the average spending per product category for each cluster. Each segment of the bar represents the average spending for a specific product category, and different colors represent different clusters. This visualization helps us understand how each cluster allocates its spending across various product categories.

In [ ]:
# Normalize category spending by total spending per cluster
normalized = df.groupby('Spending_Cluster')[
    ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
     'MntSweetProducts', 'MntGoldProds']
].mean()

# Add total spending per cluster
normalized['Total'] = normalized.sum(axis=1)

# Divide each category by total to get proportions
normalized_pct = normalized.div(normalized['Total'], axis=0).drop(columns='Total')

# Optional: Multiply by 100 for percentage
normalized_pct *= 100

print(normalized_pct.round(2))

# plot the normalized dta
# Create a colorful stacked bar chart with improved styling
plt.figure(figsize=(12, 8))

# Use a colorful palette
colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0', '#ffb3e6']
normalized_pct.plot(
    kind='bar',
    stacked=True,  # Use stacked bars to show composition
    figsize=(12, 8),
    color=colors,
    width=0.7,  # Slightly thinner bars
    edgecolor='white'  # White edge for contrast between segments
)

# Enhance styling
plt.title('Spending Composition by Customer Segment', fontsize=16, pad=20)
plt.xlabel('Customer Segment', fontsize=14, labelpad=10)
plt.ylabel('Percentage of Total Spending (%)', fontsize=14, labelpad=10)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(fontsize=12)

# Add percentage signs to y-axis
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:.0f}%'.format(x) for x in current_values])

# Improve legend
plt.legend(
    title='Product Category',
    title_fontsize=12,
    fontsize=10,
    loc='upper right',
    bbox_to_anchor=(1.15, 1)
)

# Add value labels on the stacked bars
for i, cluster in enumerate(normalized_pct.index):
    cumulative = 0
    for j, col in enumerate(normalized_pct.columns):
        value = normalized_pct.loc[cluster, col]
        if value > 5:  # Only show labels for segments > 5%
            plt.text(
                i,
                cumulative + value / 2,  # Center of segment
                f"{value:.1f}%",
                ha='center',
                va='center',
                fontsize=9,
                fontweight='bold',
                color='black' if value > 15 else 'white'  # Better contrast
            )
        cumulative += value

# Add grid lines for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Rename x-axis labels to more meaningful names
plt.gca().set_xticklabels([
    'Segment 0',
    'Segment 1',
    'Segment 2'
])

plt.tight_layout()
plt.show()
